# How to create the Media Services client and list accounts

This notebook shows you how to import packages, authenticate, and create a client for Media Services. 
It also demonstrates how to list accounts in a resource group under your subscription. 


The same exact code for this notebook is also available in the "create_client_and_auth.csx" file that you can load into any notebook to make it easier to get started quickly. You will notice this throughout other example notebooks. It makes it easier to read and just use the CSharp script load magic word to reference the script.

```csx
#load "scripts/create_client_and_auth.csx"
```


### Install packages from Nuget for using Media Services

For more details see [Importing Packages](https://github.com/dotnet/interactive/blob/main/samples/notebooks/csharp/Docs/Importing-packages.ipynb)


In [ ]:
#r "nuget:Microsoft.Azure.Management.Media, 5.0.0"
#r "nuget:Microsoft.Extensions.Configuration"
#r "nuget:Microsoft.Identity.Client"
#r "nuget:Microsoft.Rest.ClientRuntime"
#r "nuget: dotenv.net, 3.0.0"

#load "scripts/create_client_and_auth.csx"

## Add Using statements

In [ ]:
using Microsoft.Azure.Management.Media;
using Microsoft.Azure.Management.Media.Models;
using Microsoft.Extensions.Configuration;
using Microsoft.Identity.Client;
using Microsoft.Rest;
using Common_Utils;
using dotenv.net;

## Load environment variables from the .env file

Create a .env file in the folder for this interactive notebook and fill out the following variables from the values in the Azure Portal for your Media Services account. 

```env
# Copy the content of this file to a file named ".env". It should be stored at the root of the repo.
# Make sure to exclude this file from GitHub checkins using a .gitignore file

# The values can be obtained from the API Access page for your Media Services account in the portal.
AADCLIENTID=00000000-0000-0000-0000-000000000000
AADSECRET=00000000-0000-0000-0000-000000000000
AADTENANTDOMAIN=microsoft.onmicrosoft.com
AADTENANTID=00000000-0000-0000-0000-000000000000
ACCOUNTNAME="amsaccount"
RESOURCEGROUP="amsResourceGroup"
SUBSCRIPTIONID=00000000-0000-0000-0000-000000000000
ARMAADAUDIENCE=https://management.core.windows.net/
ARMENDPOINT=https://management.azure.com/
```


In [ ]:
dotenv.net.DotEnv.Load();
var envVars = dotenv.net.DotEnv.Read();
var AADCLIENTID= envVars["AADCLIENTID"];
var AADSECRET= envVars["AADSECRET"];
var AADTENANTDOMAIN = envVars["AADTENANTDOMAIN"];
var AADTENANTID= envVars["AADTENANTID"];
var ACCOUNTNAME= envVars["ACCOUNTNAME"];
var RESOURCEGROUP= envVars["RESOURCEGROUP"];
var SUBSCRIPTIONID= envVars["SUBSCRIPTIONID"];
var ARMAADAUDIENCE= envVars["ARMAADAUDIENCE"];
var ARMENDPOINT= envVars["ARMENDPOINT"];
var AADENDPOINT= envVars["AADENDPOINT"];
var STORAGECONNECTIONSTRING= envVars["STORAGECONNECTIONSTRING"];


## Authenticate using the client id and client secret

In [ ]:
string TokenType = "Bearer";
var scopes = new[] { ARMAADAUDIENCE + ".default" };

var app = ConfidentialClientApplicationBuilder.Create(AADCLIENTID)
    .WithClientSecret(AADSECRET)
    .WithAuthority(AzureCloudInstance.AzurePublic, AADTENANTID)
    .Build();

var authResult = await app.AcquireTokenForClient(scopes)
                                        .ExecuteAsync()
                                        .ConfigureAwait(false);

var credentials = new TokenCredentials(authResult.AccessToken, TokenType);


## Create the Azure Media Services Client

In [ ]:

IAzureMediaServicesClient client;

try
{
    client =new AzureMediaServicesClient(new Uri(ARMENDPOINT), credentials)
    {
        SubscriptionId = SUBSCRIPTIONID
    };
}
catch (Exception e)
{
    display("TIP: Make sure that you have filled out an .env file before running this sample.");
    display($"{e.Message}");
}

 // Set the polling interval for long running operations to 2 seconds.
// The default value is 30 seconds for the .NET client SDK
client.LongRunningOperationRetryTimeout = 2;

display("Client created for Media Services");
display($"Using AMS REST API version: {client.ApiVersion}");


## List all of the media accounts in this Resource Group

In [ ]:
display("Listing all Media accounts in this Resource Group");

IPage<MediaService> accounts;
try{
    accounts = await client.Mediaservices.ListAsync(RESOURCEGROUP);
}catch(ErrorResponseException  e)
{
    display(e.Body);
}

display(accounts);
